# MySQL connector para Python

- [`MySQL connector`](https://dev.mysql.com/doc/connector-python/en/) es una API que cumple con especificación Python Database API (PEP 249).
- [PEP 249](https://www.python.org/dev/peps/pep-0249/) fomenta la similitud entre los módulos de Python que se utilizan para acceder a las bases de datos... 
- Escrito en Python puro y no tiene dependencia excepto la Python Standard Library.

[Descargar MySQL connector](https://dev.mysql.com/downloads/connector/python/).

## Conexión con el servidor

El constructor ```connect()``` crea una conexión al servidor MySQL y retorna un objeto ```MySQLConnection```.

- [Argumentos de la conexión](https://dev.mysql.com/doc/connector-python/en/connector-python-connectargs.html).

Para tratar los errores de conexión, es posible utilizar la sentencia ```try-except``` y detectar todos los errores utilizando las excepciones [```errors.Error```](https://dev.mysql.com/doc/connector-python/en/connector-python-api-errors-error.html).

En el siguiente ejemplo, se realiza una conexión al servidor local (`localhost`) y a la base de datos `universidad`. Si la conexión se logra establecer, entonces, se procede a cerrar dicha conexión.

In [1]:
from mysql.connector import connect
from mysql.connector import Error
from mysql.connector import errorcode

try:
    cnx = connect(user='root', password='mysqlroot', host='localhost', database='universidad')
    print('Conexión establecida con exito!')
    # logica SQL
    cnx.close()
    print('Conexión cerrada!')
except Error as e:
    if e.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print('Algo anda mal con tu usuario o contraseña.')
    elif e.errno == errorcode.ER_BAD_DB_ERROR:
        print('La base de datos no existe.')
    else:
        print(e)

Conexión establecida con exito!
Conexión cerrada!


## Consultar datos

La clase `MySQLCursor` instancia un __objeto que ejecuta operaciones utilizando sentencias `SQL`__, este, interactua con el servidor utilizando un objeto `MySQLConnection`.

Para crear un cursor, se utiliza el método ```cursor()``` del objeto ```MySQLConnection```. 
- Por defecto, __el cursor es un objeto iterable de filas como tuplas__.
- Opcionalmente, pasando como argumento, el parámetro ```dictionary=True```, el objeto se convierte en un iterador de diccionarios, que utiliza como claves los nombres de los campos de la base de datos.

Para los parámetros,
- `dictionary=True` retorna filas como diccionario.
- `tuple=True`, retorna filas como tuplas.

In [2]:
# abre una conección al servidor MySQL y la almacena en la variable cnx
cnx = connect(user='root', password='mysqlroot', host='localhost', database='universidad')

# crea un cursor MySQLCursor
cursor = cnx.cursor()

# prepara la consulta
query = ('SELECT * from alumno')

# ejecuta la consulta y retorna los valores
cursor.execute(query)

for rut, nombre in cursor:
    print('{} : {}'.format(rut, nombre))

cursor.close()
cnx.close()

14652985-4 : manuel resto
14852654-7 : monica soto
15126954-8 : margarita pinilla
15785365-8 : marco ferj


__NOTA__: En el ejemplo se utiliza la base de datos `universidad`.

En el siguiente ejemplo, se ejecuta la calusula [`INNER JOIN`](https://www.w3schools.com/sql/sql_join_inner.asp) para extraer los profesores que actualmente dictan las asignaturas registradas:

In [3]:
cnx = connect(user='root', password='mysqlroot', host='localhost', database='universidad')

cursor = cnx.cursor()

query = ('''SELECT asignatura.nombre, profesor.nombre 
            FROM asignatura INNER JOIN profesor 
            ON asignatura.rutProf = profesor.rutProf''')

# ejecuta la consulta y retorna los valores
cursor.execute(query)

for asignatura, profesor in cursor:
    espacio = profesor.index(' ')
    nombre = profesor[:espacio]
    apellido = profesor[espacio+1:]
    print('{:11}: {} {}'.format(asignatura.upper(), nombre.capitalize(), apellido.capitalize()))

cursor.close()
cnx.close()

LOGISTICA  : Juan Perez
CALCULO I  : Jose Carrasco
FISICA I   : Manuel Osess


## Crear tablas

Por lo general, las tablas de una base de datos son objetos permanentes y es posible trabajar con tablas ya creadas, en lugar de crearlas. No obstante, sentecias DDL (*Data Definition Language*) se pueden ejecutar a partir de un objetos cursor.

**NOTA: Se recomienda evitar la continua creación de tablas, ya que es una operación costosa.**

En el siguiente ejemplo se crea la base de datos `empleados`.

In [13]:
DB_NAME = 'employees'
TABLES = {}
TABLES['employees'] = (
    "CREATE TABLE `employees` ("
    "`emp_no` int(11) NOT NULL AUTO_INCREMENT,"
    "`birth_date` date NOT NULL,"
    "`first_name` varchar(14) NOT NULL,"
    "`last_name` varchar(16) NOT NULL,"
    "`gender` enum('M','F') NOT NULL,"
    "`hire_date` date NOT NULL,"
    "PRIMARY KEY (`emp_no`)"
    ") ENGINE=InnoDB")

TABLES['departments'] = (
    "CREATE TABLE `departments` ("
    "`dept_no` char(4) NOT NULL,"
    "`dept_name` varchar(40) NOT NULL,"
    "PRIMARY KEY (`dept_no`), UNIQUE KEY `dept_name` (`dept_name`)"
    ") ENGINE=InnoDB")

TABLES['dept_emp'] = (
    "CREATE TABLE `dept_emp` ("
    "`emp_no` int(11) NOT NULL,"
    "`dept_no` char(4) NOT NULL,"
    "`from_date` date NOT NULL,"
    "`to_date` date NOT NULL,"
    "PRIMARY KEY (`emp_no`,`dept_no`), KEY `emp_no` (`emp_no`),"
    "KEY `dept_no` (`dept_no`),"
    "CONSTRAINT `dept_emp_ibfk_1` FOREIGN KEY (`emp_no`) "
    "REFERENCES `employees` (`emp_no`) ON DELETE CASCADE,"
    "CONSTRAINT `dept_emp_ibfk_2` FOREIGN KEY (`dept_no`) "
    "REFERENCES `departments` (`dept_no`) ON DELETE CASCADE"
    ") ENGINE=InnoDB")

La función ```create_database(cursor, db)``` que se define a continuación, crea una base de datos a partir de un cursor `MySQLCursor`.
- Recibe como argumentos __el cursor__ que contiene la sentencia `SQL` (DDL) y el __nombre de la base de datos__.
- Incluye manejo de excepciones para la ejecución de lenguaje `SQL`.

In [14]:
def create_database(cursor, db_name):
    try:
        cursor.execute("CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(db_name))
    except Error as e:
        print('Error al crear la base de datos: {}'.format(e))

En el siguiente código, se emplea la sentencia `try-except` que intenta conectarse a la base de datos de nombre `DB_NAME`. 
- Es el caso que suceda un error de código [`ER_BAD_DB_ERROR`](https://dev.mysql.com/doc/refman/5.7/en/error-messages-server.html#error_er_bad_db_error) (`ERROR 1049 (42000): Unknown database`), es decir, que no existe, se ejecuta la función `create_database()` que crea la dabe de datos.

In [17]:
cnx = connect(user='root', password='mysqlroot', host='localhost')
cursor = cnx.cursor()

# conectándose a la base de datos
try:
    cursor.execute("USE {}".format(DB_NAME))
    print("Base de datos {} seleccionada.".format(DB_NAME))
except Error as e:
    if e.errno == errorcode.ER_BAD_DB_ERROR:
        print("La base de datos {} no existe.".format(DB_NAME))
        create_database(cursor, DB_NAME)
        print("Base de datos {} fue creada exitosamente.".format(DB_NAME))
    elif e.errno == errorcode.ER_DB_EXISTS_ERROR:
        print('ya existe.')
    else:
        print(e)

Base de datos employees seleccionada.


Una vez que se crea la base de datos, o si existe, se cambia a la base de datos objetivo, se crean las tablas iterando sobre los items del diccionario `TABLES`.

In [18]:
cursor.execute("USE {}".format(DB_NAME))
for tablename, ddlcode in TABLES.items():
    try:
        print('Creando tabla {}: '.format(tablename), end='')
        cursor.execute(ddlcode)
        print('OK')
    except Error as e:
        if e.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print('Tabla {} ya existe.'.format(tablename))
        else:
            print(e.msg)

Creando tabla employees: OK
Creando tabla departments: OK
Creando tabla dept_emp: OK


## Insertar datos

En el siguiente ejemplo se insertan nuevos datos en la tabla `employees`.

In [19]:
from datetime import date, datetime, timedelta

cnx = connect(user='root', password='mysqlroot', host='localhost', database='employees')
cursor = cnx.cursor()

tomorrow = datetime.now().date() + timedelta(days=1)
add_employee = ("INSERT INTO employees "
               "(first_name, last_name, hire_date, gender, birth_date) "
               "VALUES (%s, %s, %s, %s, %s)")
data_employee = ('Juan', 'Perez', tomorrow, 'M', date(1977, 6, 14))

# insertar nuevo registro
cursor.execute(add_employee, data_employee)

# asegurarse que los datos estén ingresados en la base de datos
cnx.commit()

cursor.close()
cnx.close()

## Actividad

Diseñe la función <code>conexionMySQL()</code> que reciba como argumento, el usuario y su contraseña, el <em>host</em> y el nombre de la base de datos, y retorne el objeto <code>MySQLConnection</code>, si la conexión fue realizada con éxito, de lo contrario, que retorne el valor <code>None</code>.